# Exercise 1: Multi-Armed Bandit

In [ ]:
from __future__ import division

import numpy as np
import os
import tensorflow as tf
import json

from unityagents import UnityEnvironment

### Hyperparameters

In [ ]:
total_episodes = 4000 # Total episodes to run environment.
summary_path = './summaries/bandit-1' # Path to save summary statistics.
learning_rate = 5e-3 # Agent's learning rate.
default_brain = "BanditBrain"

### Load the Unity Environment

In [ ]:
env = UnityEnvironment("./envs/Bandit")
default_brain = env.brain_names[0]

### Examine the state space

In [ ]:
brain = env.reset()
bandit_brain = brain[default_brain]
print(bandit_brain.vector_observations)

The environment is stateless.

### The Stateless Agent

In [ ]:
class Agent(object):
    def __init__(self, learning_rate, num_actions):
        # These two lines established the feed-forward part of the network. 
        self.value_estimates = tf.Variable(tf.ones([num_actions]))
        self.action_probabilities = tf.nn.softmax(self.value_estimates / 0.5)

        # These lines establish the training proceedure. 
        # We feed the reward and chosen action into the network
        # to compute the loss, and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
        self.update_scale = tf.placeholder(shape=[1],dtype=tf.float32)

        self.responsible_value = tf.slice(self.value_estimates,self.action_holder,[1])
        
        # We take the difference between the emperical reward and the value estimate
        self.loss = tf.squared_difference(self.responsible_value, self.reward_holder)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        self.update = optimizer.minimize(self.update_scale * self.loss)

### Training the Agent

In [ ]:
if not os.path.exists(summary_path):
    os.makedirs(summary_path)

# Create our tensorflow agent
tf.reset_default_graph()
agent = Agent(learning_rate, env.brains[default_brain].vector_action_space_size)
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)
summary_writer = tf.summary.FileWriter(summary_path)

i = 0
rewards = []
losses = []
env.reset()

while i < total_episodes:
    if i < total_episodes / 2:
        # Pick action according to Boltzmann distribution.
        actions, values = sess.run([agent.action_probabilities, agent.value_estimates])
        a = np.random.choice(actions,p=actions)
        action = np.argmax(actions == a)
    else:
        # Pick action greedily
        actions, values = sess.run([agent.action_probabilities, agent.value_estimates])
        action = np.argmax(actions)

    #Get our reward from picking one of the chests.
    brains = env.step(vector_action = action, text_action = json.dumps(values.tolist()))
    bandit_brain = brains[default_brain]
    reward = bandit_brain.rewards[0]
    rewards.append(reward)

    #Update the agent.
    _, value_loss = sess.run([agent.update, agent.loss], feed_dict={agent.reward_holder:[reward], 
                                          agent.action_holder:[action],
                                        agent.update_scale:[1.0 - (i/total_episodes)]})

    losses.append(value_loss)

    #Update our running tally of scores.
    if i % 50 == 0 and i > 0:
        summary = tf.Summary()
        summary.value.add(tag='Info/Reward', simple_value=float(np.mean(rewards[-50:])))
        summary.value.add(tag='Info/Value Loss', simple_value=float(np.mean(losses[-50:])))
        summary_writer.add_summary(summary, i)
        summary_writer.flush()
        print("Trial: {}, Mean Reward: {}".format(str(i), str(round(np.mean(rewards[-50:]), 3))))
    i += 1
env.close()

In [ ]:
env.close()